# Eigenmodes of the hessian of the energy {#sec-20230528190001}

This notebook uses $\LaTeX\newcommand{\D}{\mathrm{d}}\newcommand{\E}{\mathcal{E}}\newcommand{\order}[2][1]{#2^{(#1)}}\newcommand{\reals}{\mathbb{R}}$ custom macros.

In this chapter, the eigenvalues $\alpha$ and eigenvectors $x$ of the hessian are expanded to second order in $\eta$

$$
\alpha = \order[0]{\alpha} + \eta \, \order[1]{\alpha} + \tfrac{1}{2}\eta^2 \, \order[2]{\alpha} + o(\eta^2)
$$

and

$$
x = \order[0]{x} + \eta \, \order[1]{x} + \tfrac{1}{2}\eta^2 \, \order[2]{x} + o(\eta^2).
$$

It will be shown that the potentially unstable eigenmodes are necessarily such that $\order[0]{\alpha} = 0$. In that case,

$$
\order[0]{x} = \order[0]{\chi}_j \, v_j
$$

and

$$
\order[1]{x} = \order[1]{\chi}_j \, v_j + \order[1]{\lambda} \, \order[0]{\chi}_j \, w_{j\lambda} + \order[0]{\chi}_j \, \order[1]{\xi}_k \, w_{jk}
$$

where the coefficients $\order[1]{\alpha}$, $\order[2]{\alpha}$, $\order[0]{\chi}_i$ and $\order[1]{\chi}_i$ solve the following equations

$$
\bigl( E_{ijk} \, \order[1]{\xi_k} + \order[1]\lambda \, \dot{E}_{ij} \bigr) \, \order[0]{\chi_j} = \order[1]\alpha \, \order[0]{\chi_i},
$$ {#eq-20230524065341}

and

$$
\begin{aligned}
\bigl[ E_{ijkl} \, \order[1]{\xi}_k \, \order[1]{\xi}_l + \order[1]{\lambda} \, \bigl( 2\dot{E}_{ijk} \, \order[1]{\xi}_k + \order[1]{\lambda} \, \ddot{E}_{ij} \bigr)+ E_{ijk} \, \order[2]{\xi}_k &\\
+ \order[2]{\lambda} \, \dot{E}_{ij} \bigr] \order[0]{\chi}_j
+ 2\bigl( E_{ijk} \, \order[1]{\xi}_k + \order[1]{\lambda} \, \dot{E}_{ij} \bigr) \, \order[1]{\chi}_j &= 2 \order[1]{\alpha} \, \order[1]{\chi}_i + \order[2]{\alpha} \, \order[0]{\chi}_i.
\end{aligned}
$$ {#eq-20230524065110}

In [1]:
#| code-fold: true

import lsk
from sympy import *

def symbol(label, index=None, order=None):
    if index is not None:
        label = "{" + label + "_{" + str(index) + "}}"
    else:
        label = "{" + label + "}"
    if order is None:
        return Symbol(label)
    else:
        return Symbol(r"{\order[" + str(order) + "]" + label + "}")

η, u_hat = symbols(r"\eta \hat{u}")

E2, E2_dot, E2_ddot = symbols(r"\E_2 \dot{\E}_2 \ddot{\E}_2")
E3, E3_dot = symbols(r"\E_3 \dot{\E}_3")
E4 = Symbol(r"\E_4")

E_dot_ij, E_ddot_ij = symbols(r"\dot{E}_{ij} \ddot{E}_{ij}")
E_ijk, E_dot_ijk = symbols("E_{ijk} \dot{E}_{ijk}")
E_ijkl = Symbol("E_{ijkl}")

λ1, λ2 = [symbol(r"\lambda", order=k) for k in [1, 2]]
α1, α2 = [symbol(r"\alpha", order=k) for k in [1, 2]]
y1, y2 = [symbol("y", order=k) for k in [1, 2]]

indices = ["i", "j", "k", "l"]
v_i, v_j, v_k, v_l = [symbol("v", i) for i in indices]
ξ1_i, ξ1_j, ξ1_k, ξ1_l = [symbol(r"\xi", index=i, order=1) for i in indices]
ξ2_i, ξ2_j, ξ2_k, ξ2_l = [symbol(r"\xi", index=i, order=2) for i in indices]
χ0_i, χ0_j, χ0_k, χ0_l = [symbol(r"\chi", index=i, order=0) for i in indices]
χ1_i, χ1_j, χ1_k, χ1_l = [symbol(r"\chi", index=i, order=1) for i in indices]
χ2_i, χ2_j, χ2_k, χ2_l = [symbol(r"\chi", index=i, order=2) for i in indices]

w_iλ, w_jλ, w_kλ, w_lλ = [symbol("w", index=iλ) for iλ in [r"i\lambda", r"j\lambda", r"k\lambda", r"l\lambda"]]
w_ij, w_ik, w_il, w_jk, w_jl, w_kl = [symbol("w", ij) for ij in ["ij", "ik", "il", "jk", "jl", "kl"]] 

In view of stability analysis, we consider in this chapter the hessian of the energy along the bifurcated branch. We derive an asymptotic expansion of its eigenmodes. The eigenvalues $\alpha \in \reals$ and eigenvectors $x \in U$ of the hessian are such that
$$
\E_{, u u} [u(\eta), \lambda(\eta); x, \hat{u}] = \alpha \, \langle x, \hat{u} \rangle \quad \text{for all} \quad \hat{u} \in U,
$$ {#eq-20230517062340}
where $\eta \mapsto \lambda(\eta)$ and $\eta \mapsto u(\eta)$ define the bifurcated branch ($\eta$: parameter, yet unspecified). We focus in this chapter on potentially unstable eigenmodes, for which the eigenvalue $\alpha$ might be negative in the vicinity of $\eta = 0$.

In @sec-20230102153124 and in @sec-20230523130742, the following asymptotic expansions of $\lambda(\eta)$, $u(\eta)$ and $\E_{,uu}[u(\eta), \lambda(\eta); \bullet, \bullet]$ were derived

In [2]:
#| code-fold: true

u1 = ξ1_k * v_k
u1_ = ξ1_l * v_l # Will be necessary below to avoid index clashes in E4 * u1 * u1
u2 = ξ2_k * v_k + ξ1_k * ξ1_l * w_kl + 2 * λ1 * ξ1_k * w_kλ

λ = η * λ1 + η ** 2 / 2 * λ2
u = η * u1 + η ** 2 / 2 * u2

E_uu0 = E2
E_uu1 = E3 * u1 + λ1 * E2_dot
E_uu2 = (E4 * u1 * u1_ + E3 * u2 + 2 * λ1 * E3_dot * u1 + λ1 ** 2 * E2_ddot + λ2 * E2_dot)
E_uu = E_uu0 + η * E_uu1 + η**2 / 2 * E_uu2
lsk.display_latex_dict({r"\lambda": λ,
                        r"\order[1]{u}": u1,
                        r"\order[2]{u}": u2,
                        r"\E_{,uu}": E_uu.expand()}, num_cols=1)

<IPython.core.display.Math object>

The eigenvalues and eigenvectors are expanded to second order in $\eta$
$$
\alpha = \order[0]\alpha + \eta \, \order[1]\alpha + \tfrac{1}{2} \eta^2 \, \order[2]\alpha + o(\eta^2)
$$
and
$$
x = \order[0]x + \eta \, \order[1]x + \tfrac{1}{2} \eta^2 \, \order[2]x + o(\eta^2).
$$

Plugging these expansions into Eq. (@eq-20230517062340) delivers 3 variational problems (of order 0, 1 and 2 in $\eta$). These variational problems are studied below successively.

## Variational problem of order 0

The lowest-order problem reads: find $\order[0]x \in U$ and $\order[0]\alpha\in\reals$ such that, for all $\hat{u} \in U$
$$
\E_2(\order[0]x, \hat{u}) = \order[0]\alpha \langle \order[0]x, \hat{u} \rangle.
$$

The above equation shows that $(\order[0]\alpha, \order[0]x)$ is an eigenpair of $\E_2$. Since $\E_2$ is positive, $\order[0]\alpha \geq 0$. If $\order[0]\alpha > 0$, then $\alpha > 0$ in the neighborhood of $\lambda_0$, and the eigenmode is stable. Potentially unstable modes are therefore such that
$\order[0]\alpha=0$. In other words, the lowest-order $\order[0]x$ belongs to $V$ and can be expressed as a linear combination of the $v_i$

In [3]:
#| code-fold: true
α0 = 0
x0 = χ0_j * v_j

lsk.display_latex_dict({
    r"\order[0]{\alpha}": α0,
    r"\order[0]x": x0})

<IPython.core.display.Math object>

## Variational problem of order 1

The first-order term $\order[1]x$ is decomposed as the sum of its projections onto $V$ and $W$

In [4]:
#| code-fold: true
x1 = χ1_j * v_j + y1
lsk.display_latex_equation(r"\order[1]x", x1)

<IPython.core.display.Math object>

where $\order[1]y \in W$. We then plug the resulting asymptotic expansions of $\alpha$ and $x$ into the asymptotic expansions of $\E_{,uu}(x, \hat{u})$ and $\alpha \, \langle x, \hat{u} \rangle$ and extract the first-order terms.

In [5]:
α = α0 + η * α1
x = x0 + η * x1

lhs = expand(E_uu * x * u_hat).coeff(η, 1)
rhs = expand(α * x * u_hat).coeff(η, 1) 

The problem reads: find $\order[1]x \in U$ and $\order[1]\alpha\in\reals$ such that, for all $\hat{u} \in U$

In [6]:
#| code-fold: true
lsk.display_latex_equation(lhs, rhs)

<IPython.core.display.Math object>

The above equation must be understood as
$$
\order[1]{\chi}_i \, \E_2(v_i, \hat{u})
+ \E_2(\order[1]{y}, \hat{u})
+ \order[0]{\chi}_j \, \order[1]{\xi}_k \, \E_3(v_j, v_k, \hat{u})
+ \order[1]{\lambda} \, \order[0]{\chi}_j \, \dot{\E}_2(v_j, \hat{u})
= \order[1]{\alpha} \, \order[0]{\chi}_j \, \langle v_j, \hat{u} \rangle,
$$
and, since the first term cancels
$$
\E_2(\order[1]{y}, \hat{u})
+ \order[0]{\chi}_j \, \order[1]{\xi}_k \, \E_3(v_j, v_k, \hat{u})
+ \order[1]{\lambda} \, \order[0]{\chi}_j \, \dot{\E}_2(v_j, \hat{u})
= \order[1]{\alpha} \, \order[0]{\chi}_j \, \langle v_j, \hat{u} \rangle,
$$ {#eq-20230523134144}

Eq. (@eq-20230523134144)  is first tested with $\hat{u} = v_i \in V$. Observing that $\langle v_i, v_j \rangle = \delta_{ij}$ since $(v_i)$ is orthonormal, we get
$$
\bigl[ \E_3(v_i, v_j, v_k) \, \order[1]{\xi_k} + \order[1]\lambda \, \dot{\E}_2(v_i, v_j)\bigr] \, \order[0]{\chi_j} = \order[1]\alpha \, \order[0]{\chi_i},
$$
which shows that $(\order[1]\alpha, \order[0]{\chi_i})$ is an eigenpair of the symmetric tensor $(E_{ijk} \order[1]{\xi_k} + \order[1]\lambda \, \dot{E}_{ij})$ – see Eq. (@eq-20230524065341).

The test function is now picked in $W = V^\perp$ and Eq. (@eq-20230523134144) results in the following variational problem: find $\order[1]y \in W$ such that, for all $\hat{w} \in W$,
$$
\E_2(\order[1]y, \hat{w}) + \E_3(v_i, v_j, \hat{w}) \, \order[0]{\chi_i} \, \order[1]{\xi_j} + \order[1]\lambda \, \dot{\E}_2(v_i, \hat{w}) \, \order[0]{\chi_i} = 0,
$$
(observe that $\langle v_j, \hat{w} \rangle = 0$ since $V$ and $W$ are orthogonal subspaces). The solution to the above problem is expressed as a linear combination of the $w_{ij}$ and $w_{i\lambda}$ –defined by the variational problems (@eq-20230107180410) and (@eq-20230107180501), respectively–, delivering

In [7]:
#| code-fold: true
y1 = χ0_j * ξ1_l * w_jl + λ1 * χ0_j * w_jλ
x1 = χ1_j * v_j + y1

lsk.display_latex_dict({
    r"\order[1]y": y1,
    r"\order[1]x": x1})

<IPython.core.display.Math object>

## Variational problem of order 2

The second-order term $\order[2]x$ is also decomposed as the sum of its projections onto $V$ and $W = V^\perp$

In [8]:
#| code-fold: true
x2 = χ2_j * v_j + y2

lsk.display_latex_equation(r"\order[2]x", x2)

<IPython.core.display.Math object>

where $\order[2]y \in W$. We then plug the resulting asymptotic expansions of $\alpha$ and $x$ into the asymptotic expansions of $\E_{,uu}(x, v_i)$ and $\alpha \, \langle x, v_i \rangle$ and extract the second-order terms. Note that the **test-function $v_i$ belongs to $V$**. The resulting variational problem reads: find $\order[1]\alpha, \order[1]{\chi}_1, \ldots, \order[1]{\chi}_m \in \reals$ and $\order[1]y \in W$ such that, for all $\hat{v} \in V$

In [9]:
#| code-fold: true
α = α0 + η * α1 + η**2 / 2 * α2
x = x0 + η * x1 + η**2 / 2 * x2

# The substitution below makes the expression nicer
lhs_raw = expand(E_uu * x * v_i).coeff(η, 2)
rhs_raw = expand(α * x * v_i).coeff(η, 2) 

lsk.display_latex_equation(lhs_raw, rhs_raw)

<IPython.core.display.Math object>

We then perform the following substitutions in the left-hand side

1. Cancellation of all terms involving $\E_2$, since all multilinear forms involved in the left-hand side are evaluated at $v_i$ and $\E_2(v_i, \bullet) = 0$.
3. Renaming of some indices
   $$\order[1]{\lambda} \, \order[0]{\chi}_j \, \order[1]{\xi}_l \, \dot{\E}_2(v_i, w_{jl}) \rightarrow \order[1]{\lambda} \, \order[0]{\chi}_j \, \order[1]{\xi}_k \, \dot{\E}_2(v_i, w_{jk})$$
2. Symmetrization of some terms
   $$\order[1]{\xi}_k \, \order[1]{\xi}_l \, v_k \, w_{jl} \rightarrow \tfrac{1}{2} \order[1]{\xi}_k \, \order[1]{\xi}_l \, \bigl(v_k \, w_{jl} + v_l \, w_{jk} \bigr)$$
3. Introduction of the quantities $E_{ij}$, $\dot{E}_{ij}$, $\ddot{E}_{ij}$, $E_{ijk}$, $\dot{E}_{ijk}$, $E_{ijkl}$ defined in @sec-20230528173310.
4. Use of equations (@eq-20230107180410) and (@eq-20230107180501) that define $w_{ij}$ and $w_{i\lambda}$
   $$\E_3(v_i, v_j, w_{k\lambda}) \rightarrow -\E_2(w_{ij}, w_{k\lambda}) \quad \text{and} \quad \dot{\E}_2(v_i, w_{j\lambda}) \rightarrow -\E_2(w_{i\lambda}, w_{j\lambda})$$
   
and in the right-hand side
1. $(v_i)$ is orthonormal
   $$\langle v_i, v_j \rangle \rightarrow \delta_{ij}$$
2. $v_i \in V$ and $w_{j\lambda}, w_{jl} \in W = V^\perp$ are orthogonal
   $$\langle v_i, w_{j\lambda} \rangle \rightarrow 0 \quad \text{and} \quad \langle v_i, w_{jl} \rangle \rightarrow 0$$
   
The variational problem of order 2 then reduces to

In [10]:
#| code-fold: true 
lhs = (2 * lhs_raw).subs({
    # Item 1
    E2: 0,
    # Item 2
    λ1 * χ0_j * ξ1_l * E2_dot * v_i * w_jl: λ1 * χ0_j * ξ1_k * E2_dot * v_i * w_jk,
    # Item 3
    χ0_j * ξ1_k * ξ1_l * E3 * v_i * v_k * w_jl : χ0_j * ξ1_k * ξ1_l * E3 * v_i * (v_k * w_jl + v_l * w_jk) / 2,
    # Item 4
    E2_dot * v_i * v_j: E_dot_ij,
    E2_ddot * v_i * v_j : E_ddot_ij - E2_dot * v_i * w_jλ - E2_dot * v_j * w_iλ,
    E3 * v_i * v_j * v_k: E_ijk,
    E3_dot * v_i * v_j * v_k : E_dot_ijk - E2_dot * v_i * w_jk - E2_dot * v_j * w_ik - E2_dot * v_k * w_ij,
    E4 * v_i * v_j * v_k * v_l: E_ijkl - E3 * v_i * v_j * w_kl - E3 * v_i * v_k * w_jl - E3 * v_i * v_l * w_jk,
    # Item 5
    E3 * v_i * v_j * w_kλ: -E2 * w_ij * w_kλ,
    E3 * v_i * v_k * w_jλ: -E2 * w_ik * w_jλ,
    E2_dot * v_i * w_jλ: -E2 * w_iλ * w_jλ,
    E2_dot * v_j * w_iλ: -E2 * w_iλ * w_jλ,
    E2_dot * v_j * w_ik: -E2 * w_jλ * w_ik,
    E2_dot * v_k * w_ij: -E2 * w_kλ * w_ij
}).expand()

rhs = (2 * rhs_raw).subs({
    v_i * w_jλ: 0,
    v_i * w_jl: 0,
    v_i * v_j * χ0_j: χ0_i,
    v_i * v_j * χ1_j: χ1_i
})

lsk.display_latex_equation(lhs, rhs)

<IPython.core.display.Math object>

which upon reordering delivers Eq. (@eq-20230524065110).